In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

X, y = make_regression(n_samples=3_000, n_features=5, n_informative=3, noise=33, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Mean Absolute Error
Средняя разница между фактическими значениями и предсказанными по модулю.\
Метрика устойчива к выбросам, однако не учитывает масштаб и направление ошибок
$$MAE = \frac{1}{N}\sum_{i=1}^N{|y_i-\hat{y_i}|}$$

In [2]:
from sklearn.metrics import mean_absolute_error

In [3]:
mean_absolute_error(y_test, y_pred)

27.050203889364692

## Mean Absolute Percentage Error
Позволяет оценить насколько в среднем прогнозы модели отличаются относительно реальных значений в процентном отношении.\
При $y$=0 выдает неадекватное значения из за нуля в знаминателе. Искажается при маленьких значениях $y$.
$$MAPE=\frac{1}{N}\sum_{i=1}^N{\frac{|y_i-\hat{y_i}|}{y_i}}$$

In [4]:
from sklearn.metrics import mean_absolute_percentage_error

In [5]:
mean_absolute_percentage_error(y_test, y_pred)

2.1721101322769907

## Symmetric Mean Absolute Percentage Error
Модификация над **MAPE** выражает ошибку прогноза в процентах, но более сбалансированно.\
Устраняет асимметрию при отклонениях от малых значений и проблему деления на ноль\
*(Но сумма двух модулей в знаминателе в некоторых случаях все равно может составить ноль)*

Не учитывает относительную значимость наблюдений и плохо работает с неравномерными данными.

$$SMAPE=\frac{1}{N}\sum_{i=1}^N{\frac{2|y_i-\hat{y_i}|}{|y_i|+|\hat{y_i}|}}$$

In [6]:
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    numerator = 2*np.abs(y_true - y_pred)
    denominator = np.abs(y_true)+np.abs(y_pred)
    smape = np.mean(numerator[denominator != 0] / denominator[denominator!=0])
    return smape

In [7]:
symmetric_mean_absolute_percentage_error(y_test, y_pred)

0.7382988418430133

## Weighted Average Percentage Error
Нормализует общую ошибку к общей сумме фактических значений \
Использует взвешенное среднее абсолютных ошибок, где вес — это размер фактических значений. \
Эффективна, когда данные с сильно разными масштабами.
$$WAPE=\frac{\sum_{i=1}^N{|y_i-\hat{y_i}|}}{\sum_{i=1}^N{|y_i|}}$$

In [9]:
def weighted_average_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    numerator = np.sum(np.abs(y_true - y_pred))
    denominator = np.sum(np.abs(y_true))
    wape = numerator/denominator
    return wape

In [10]:
weighted_average_percentage_error(y_test, y_pred)

0.40434787506534264

# Mean Squared Error
Средняя разница между фактическими значениями и предсказанными в квадрате.\
Метрика делает акцент на больших ошибках, чувствительна к выбросам.
$$MSE = \frac{1}{N}\sum_{i=1}^N{(y_i-\hat{y_i})^2}$$
Извлечение квадратного корня возвращает значение к исходной размерности и улучшает интепритируемость.
$$RMSE = \sqrt{\frac{1}{N}\sum_{i=1}^N{(y_i-\hat{y_i})^2}}$$

In [14]:
from sklearn.metrics import mean_squared_error, root_mean_squared_error
display(
    mean_squared_error(y_test, y_pred),
    root_mean_squared_error(y_test, y_pred)
)

1121.915175038442

33.49500223971394

## Mean Squared Logarithmic Error
Альтернативный способ перехода от абсолютных ошибок к относительным - измерение в логорифмическом масштабе.\
Метрика может быть применима лишь для неотрицательных меток

Константа $c$ - вводится искуственно для избегания логорифма от нуля.\
Устойчива к выбросам, поскольку делает распределение целевых и спрогнозированных значений более однородным. \
Труднее интерпретировать, уделяет больше внимания заниженным прогнозам, поскольку логарифм является несимметричной функцией.
$$MSLE = \frac{1}{N}\sum_{i=1}^N{(ln(y_i+c)-ln(\hat{y_i}+c))^2}$$
$$RMSLE = \sqrt{\frac{1}{N}\sum_{i=1}^N{(ln(y_i+c)-ln(\hat{y_i}+c))^2}}$$

In [17]:
from sklearn.metrics import mean_squared_log_error, root_mean_squared_log_error

# $R^2$ score
Коэффициент детерминации - показывает какая доля дисперсии(разброса) объясняется моделью\
Не учитывает количество признаков в модели. Имеет тенденцию к увеличению при добавлении новых признаков в обучающий набор, даже если они не улучшают качество модели. Сравнение моделей с разным количеством признаков становится некорректным
$$R^2=1-\frac{RSS}{TSS}$$
$$TSS = \sum_{i=1}^n (y-\bar{y})^2 \hspace{5cm} RSS = \sum_{i=1}^n (y-\hat{y})^2$$

$TSS$ - **Total Sum of Squares** Общая сумма квадратов отклонений от среднего.\
$RSS$ - **Residual Sum of Squares** Сумма квадратов остатков\
$\bar{y}$ - Общее реднее значение выборки\
$\hat{y}$ - Предсказанное значение\


In [19]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.8456162138953873

## Adjusted $R^2$ score
Скорректированный коэффициент детерминации. Вводит штраф за добавление новых признаков.
$$R^2_{adj}=1-\frac{RSS(n-1)}{TSS(n-k-1)}=1-\frac{(1-R^2)(n-1)}{n-k-1}$$
$k$ - Количество признаков.\
$n$ - Количество образцов.

In [22]:
def adjusted_r2_score(y_true, y_pred, X_shape):
    n, k = X_shape
    RSS = np.sum((y_true - y_pred)**2)
    TSS = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - RSS/TSS
    return 1 - ((1-r2)*(n-1)) / (n-k-1)

In [21]:
adjusted_r2_score(y_test, y_pred, X_test.shape)

0.8443166870763249

# Bias-Variance Decomposition

**Bias** - Смещение. Недообученность модели. Прогнозы стабильно в среднем отклоняются от истины. \
**Variance** - Разброс. Переобученность модели. Прогнозы разбросаны вокруг истины. Модель подражает конкретной выборке, не обобщается.

**Biance-Variance trade-off** - Поиск наиболее сбалансированной модели между смещением и разбросом.\
Увеличение сложности модели -> уменьшается смещение, растет разброс\
Упрощение модели -> увеличивает смещение, уменьшает разброс

**MSE** для образца $x$ — это среднее значение квадрата разности между истинным $y=f(x)+\epsilon$ и предсказанием модели $\hat{f}(x)$ если усреднить по всем возможным обучающим выборкам и всем возможным шумам в данных.
$$
MSE(x) = \mathbb{E}_{D, \epsilon} [(y-\hat{f}(x))^2]
$$
$x$ - Образец данных\
$y=f(x)+\epsilon$ - Истинный закон + случайный шум\
$\hat{f}(x))$ - Прогноз модели, обученной на случайной выборке из $D$\
$\mathbb{E}_{D, \epsilon}$ - Математическое ожидание по выборке и шуму\
$\epsilon$ - Случайный шум с $\mathbb{E}[\epsilon]=0$ и  $Var(\epsilon) =\mathbb{E}[\epsilon^2] = \sigma^2$

$$MSE(x) = \mathbb{E}_{D, \epsilon} [(f(x)+\epsilon-\hat{f}(x))^2] = $$
$$=\mathbb{E}_{D, \epsilon} \Big[\Big(\big(f(x)-\hat{f}(x)\big)+\epsilon\Big)^2\Big] = $$
$$= \mathbb{E}_{D, \epsilon}\Big[  \big(f(x)-\hat{f}(x)\big)^2     +2\big(f(x)-\hat{f}(x)\big)\epsilon        + \epsilon^2  \Big] =$$

$$ =\mathbb{E}_{D, \epsilon}\Big[  \big(f(x)-\hat{f}(x)\big)^2\Big] + \mathbb{E}_{D, \epsilon}\Big[ 2\big(f(x)-\hat{f}(x)\big)\epsilon \Big]         + \mathbb{E}_{D, \epsilon}\Big[ \epsilon^2\Big] =  $$

**Третий член**\
$\mathbb{E_\epsilon}[\epsilon^2] = \sigma^2$


**Второй член**\
$\epsilon$ и $\hat{f}(x)$ независимы(шум появляется после обучения модели), $\mathbb{E}[\epsilon]=0$\
$\mathbb{E}_{D, \epsilon}\Big[ 2\big(f(x)-\hat{f}(x)\big)\epsilon \Big]         =   \mathbb{E}_{D}\Big[ (f(x)-\hat{f}(x)) * \underbrace{\mathbb{E}[\epsilon}_0] \Big]    =0$

**Разложение дисперсии**

Для случайной величины $Z$ и ее математического ожидания $\mathbb{E}[Z]$:

$$
\mathbb{E}[(Z - \mu)^2] = Var(Z) + Bias(Z)^2
$$

$$
Var(Z) = \mathbb{E}[(Z - \mathbb{E}[Z])^2] 
$$
$$
Bias(Z)^2 = (\mathbb{E}[Z]-\mu)^2
$$





$Z$ - Некая случайная величина\
$\mu$ - Истинное значение, с которым сравнивается $Z$







---



$$MSE(x) = \mathbb{E}_D  \Big[    \Big(f(x)-\hat{f}(x)\Big)^2  \Big] + \sigma^2$$

$$
MSE(x) = \underbrace{\mathbb{E}_D [(\hat{f}(x) - \mathbb{E}_D[\hat{f}(x)])^2]}_{variance}     +    \underbrace{(\mathbb{E}[\hat{f}(x)]- f(x))^2}_{bias^2} + \underbrace{\sigma^2}_{noise}
$$
$$
MSE(x) = Var[\hat{f}(x)] + Bias[\hat{f}(x)]^2 + \sigma^2
$$
